In [3]:
# Import necessary modules and plug-ins
import numpy as np
import pandas as pd
import strawberryfields as sf
import thewalrus as tw
import pennylane as qml

from strawberryfields import ops
from strawberryfields.ops import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from strawberryfields.utils import random_interferometer

In [4]:
# Pinging Strrawberry Fields to establish connection & security
sf.ping()

You have successfully authenticated to the platform!


In [5]:
eng = sf.RemoteEngine("X8")
engtoo = sf.Engine("gaussian")

In [6]:
# Set the random seed
np.random.seed(42)

In [7]:
# Defining the linear inferometer
U = np.array([
 [ 0.219546940711-0.256534554457j, 0.611076853957+0.524178937791j,
    -0.102700187435+0.474478834685j,-0.027250232925+0.03729094623j],
 [ 0.451281863394+0.602582912475j, 0.456952590016+0.01230749109j,
    0.131625867435-0.450417744715j, 0.035283194078-0.053244267184j],
 [ 0.038710094355+0.492715562066j,-0.019212744068-0.321842852355j,
    -0.240776471286+0.524432833034j,-0.458388143039+0.329633367819j],
 [-0.156619083736+0.224568570065j, 0.109992223305-0.163750223027j,
    -0.421179844245+0.183644837982j, 0.818769184612+0.068015658737j]
])

In [8]:
# Defining another but randomly
Utoo = random_interferometer(4)
np.set_printoptions(precision=16)
print(Utoo)

[[ 0.2364882618857422-0.482214307322667j
   0.0682964773269564+0.0444789775762852j
   0.5115007388835463-0.0952986583732719j
   0.5520571865947266-0.3597469917942807j]
 [-0.1114816726427168+0.6978032089401686j
  -0.2494382786327934+0.0841070110434553j
   0.467059291172189 -0.4319298125262462j
   0.1622065409340211-0.0181760216390895j]
 [-0.2235192632554648-0.2591835234472852j
   0.24364996116129  -0.0537562295998913j
  -0.0925982940067332-0.5381058778231553j
   0.2726770843179723+0.6694197728029507j]
 [ 0.1151995301476408-0.2859672891120847j
  -0.9016492271986603-0.2209918615739196j
  -0.0962775827896406-0.1310559524249577j
  -0.0200152048755105+0.127661278317911j ]]


In [9]:
# initialize a 4 mode program
firstprog = sf.Program(8, name="first_remote_job")

In [10]:
with firstprog.context as q:
    # Initial squeezed states
    # Allowed values are r=1.0 or r=0.0
    ops.S2gate(1.0) | (q[0], q[4])
    ops.S2gate(1.0) | (q[1], q[5])
    ops.S2gate(1.0) | (q[3], q[7])

    # Interferometer on the signal modes (0-3)
    ops.Interferometer(U) | (q[0], q[1], q[2], q[3])
    ops.BSgate(0.543, 0.123) | (q[2], q[0])
    ops.Rgate(0.453) | q[1]
    ops.MZgate(0.65, -0.54) | (q[2], q[3])

    # *Same* interferometer on the idler modes (4-7)
    ops.Interferometer(U) | (q[4], q[5], q[6], q[7])
    ops.BSgate(0.543, 0.123) | (q[6], q[4])
    ops.Rgate(0.453) | q[5]
    ops.MZgate(0.65, -0.54) | (q[6], q[7])

    ops.MeasureFock() | q

In [11]:
secondprog = sf.Program(8, name="second_remote_job")

In [12]:
with secondprog.context as q:
    # Initial squeezed states
    # Allowed values are r=1.0 or r=0.0
    ops.S2gate(1.0) | (q[0], q[4])
    ops.S2gate(1.0) | (q[1], q[5])
    ops.S2gate(1.0) | (q[3], q[7])

    # Interferometer on the signal modes (0-3)
    ops.Interferometer(Utoo) | (q[0], q[1], q[2], q[3])
    ops.BSgate(0.543, 0.123) | (q[2], q[0])
    ops.Rgate(0.453) | q[1]
    ops.MZgate(0.65, -0.54) | (q[2], q[3])

    # *Same* interferometer on the idler modes (4-7)
    ops.Interferometer(Utoo) | (q[4], q[5], q[6], q[7])
    ops.BSgate(0.543, 0.123) | (q[6], q[4])
    ops.Rgate(0.453) | q[5]
    ops.MZgate(0.65, -0.54) | (q[6], q[7])

    ops.MeasureFock() | q

In [33]:
# can only run on real hardware monday-thursday 8-4pm ET
results_first_job = eng.run(firstprog, shots=8, cutoff_dim=25)

In [34]:
results_second_job = eng.run(secondprog, shots=8, cutoff_dim=25)

In [35]:
pd.DataFrame(results_first_job.samples).style

,0,1,2,3,4,5,6,7
0,0,0,0,0,2,0,1,0
1,0,0,0,0,2,2,0,0
2,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,1,0
4,0,0,0,0,0,0,0,0
5,0,0,0,1,1,0,0,0
6,0,0,0,0,0,0,1,0
7,0,0,1,0,0,1,2,0


In [36]:
pd.DataFrame(results_second_job.samples).style

,0,1,2,3,4,5,6,7
0,1,0,0,0,0,0,0,0
1,1,0,1,0,1,0,0,0
2,0,0,1,1,0,0,0,1
3,0,1,1,0,0,0,1,0
4,0,0,0,0,1,0,0,0
5,0,1,0,0,0,0,0,2
6,0,0,0,0,0,0,1,0
7,2,2,0,0,1,1,0,1


In [58]:
pd.DataFrame(tw.quantum.Amat(results_first_job.samples)).style

,0,1,2,3,4,5,6,7
0,0.126582+1.058019j,-0.162098-0.809505j,-0.185556+0.307484j,-0.005543+0.074312j,1.420425-0.050927j,-0.524287+0.709344j,0.282152-0.054352j,0.102537+0.044176j
1,0.130579-0.471563j,-0.208364+1.358741j,0.236400-0.195475j,0.052980-0.028499j,-0.170901-0.539936j,1.865804+0.038050j,-0.162957-0.048961j,-0.109978-0.077460j
2,-389259478522327.125000-730119206088619.375000j,229480547588456.375000+673772335531162.125000j,364252940528375.500000+2176465097158108.000000j,-214661858406855.375000-1530866196905673.500000j,716226684980868.875000+414266016516278.000000j,-665436822866512.500000-252634749434707.000000j,-2162572576050357.000000-439272554510228.625000j,1522530684241023.500000+267453438616306.687500j
3,730119206088620.250000-389259478522326.875000j,-673772335531162.875000+229480547588455.812500j,-2176465097158109.500000+364252940528376.000000j,1530866196905674.000000-214661858406855.562500j,-414266016516277.937500+716226684980868.875000j,252634749434706.812500-665436822866512.250000j,439272554510228.937500-2162572576050357.750000j,-267453438616306.812500+1522530684241024.500000j
4,1.376689+0.103885j,-0.543708-0.777872j,0.370777-0.025338j,0.057010-0.001689j,0.046453-1.190034j,-0.185811+0.822635j,-0.145270-0.074747j,-0.113176-0.179054j
5,-0.128378+0.479730j,1.853041-0.006757j,-0.304054+0.175676j,-0.020270+0.003378j,0.219595+0.567568j,-0.190878-1.395270j,0.165541-0.006757j,0.101351+0.108108j
6,-389259478522326.500000-730119206088620.000000j,229480547588455.625000+673772335531162.875000j,364252940528375.750000+2176465097158108.500000j,-214661858406855.500000-1530866196905673.500000j,716226684980868.500000+414266016516277.875000j,-665436822866511.750000-252634749434706.500000j,-2162572576050357.500000-439272554510228.750000j,1522530684241024.000000+267453438616307.125000j
7,-730119206088619.750000+389259478522327.000000j,673772335531162.375000-229480547588456.062500j,2176465097158108.750000-364252940528375.875000j,-1530866196905673.750000+214661858406855.218750j,414266016516278.000000-716226684980868.625000j,-252634749434707.062500+665436822866512.375000j,-439272554510229.125000+2162572576050357.000000j,267453438616306.625000-1522530684241023.750000j


In [57]:
pd.DataFrame(tw.quantum.Amat(results_second_job.samples)).style

,0,1,2,3,4,5,6,7
0,-0.000000+0.250000j,-0.500000-0.500000j,-0.000000+0.000000j,0.500000+0.000000j,0.000000+0.250000j,0.500000-0.500000j,0.000000-0.000000j,-0.500000+0.000000j
1,0.500000+2.000000j,-4.000000-3.000000j,-2.000000-2.000000j,4.000000+3.000000j,0.500000+2.000000j,3.000000-5.000000j,2.000000-2.000000j,-4.000000+5.000000j
2,-0.250000-1.250000j,0.000000+1.000000j,1.000000-0.000000j,-0.500000-0.500000j,0.750000-0.250000j,-1.000000+2.000000j,0.000000+0.000000j,0.500000-1.500000j
3,-1.500000+0.000000j,5.000000+1.000000j,1.000000-0.000000j,-4.000000-1.000000j,-1.500000-2.000000j,1.000000+3.000000j,-1.000000+2.000000j,1.000000-3.000000j
4,0.000000-0.250000j,0.500000+0.500000j,0.000000-0.000000j,-0.500000+0.000000j,-0.000000-0.250000j,-0.500000+0.500000j,-0.000000+0.000000j,0.500000-0.000000j
5,0.500000-2.000000j,3.000000+5.000000j,2.000000+2.000000j,-4.000000-5.000000j,0.500000-2.000000j,-4.000000+3.000000j,-2.000000+2.000000j,4.000000-3.000000j
6,0.750000+0.250000j,-1.000000-2.000000j,-0.000000+0.000000j,0.500000+1.500000j,-0.250000+1.250000j,-0.000000-1.000000j,1.000000-0.000000j,-0.500000+0.500000j
7,-1.500000+2.000000j,1.000000-3.000000j,-1.000000-2.000000j,1.000000+3.000000j,-1.500000+0.000000j,5.000000-1.000000j,1.000000-0.000000j,-4.000000+1.000000j
